In [1]:
from datasets import Dataset, Image, load_dataset
import os
from transformers import pipeline
import tqdm
import numpy as np

/home/jbk/miniconda3/envs/cn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Test Blip

In [3]:
captioner = pipeline("image-to-text",model="Salesforce/blip-image-captioning-large", device=0)
def caption_image_data(example):
    image = example["input"]
    image_caption = captioner(image)[0]['generated_text']
    example['image_caption'] = image_caption
    return example
image_paths = [os.path.join(path, file) for path, subdirs, files in os.walk("../../data/samples") for file in files if file.endswith(".jpg")]
sample_set = Dataset.from_dict({'input' : image_paths}).cast_column("input", Image())
caption_set = sample_set.map(caption_image_data)
for example in tqdm.tqdm(caption_set):
    print(example['image_caption'])

Map:   0%|          | 0/7 [00:00<?, ? examples/s]/home/jbk/miniconda3/envs/cn/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 7/7 [00:00<00:00, 323.58it/s]

a view of a lake and a mountain from space
a view of a satellite image of a mountain range
there is a picture of a snowboarder riding down a snowy hill
arafed view of a mountain with a few clouds in the sky
a close up of a picture of a snow covered area
a view of a large body of water with a cloud in the sky
a close up of a satellite image of a rusted area


# Extract Bits

In [20]:
bit_image = np.load('../../data/samples/tsts.npy')
bit_image.shape
# Clouds
(np.right_shift(bit_image,3)&1).sum()
# Snow
(np.right_shift(bit_image,5)&1).sum()

0

## Convert Folder into Captions

In [18]:
src_dir = '../../data/samples'
target_dir = '../../data/captions/'

cloud_modifiers = ["clear", "slightly cloudy", "cloudy", "very cloudy"]
snow_modifiers = ["", " and there is some snow", " and it is snowy", " and it is very snowy"]
caption_prompt = "A satellite image of the earth. The weather is {cloud}{snow}."
captions = []


def get_cloud_segment(bit_image):
    cloud_pixels = (np.right_shift(bit_image,3)&1).sum()
    cloud_ratio = cloud_pixels / bit_image.size
    if cloud_ratio < 0.1:
        return 0
    elif cloud_ratio < 0.3:
        return 1
    elif cloud_ratio < 0.5:
        return 2
    else:
        return 3

def get_snow_segment(bit_image):
    snow_pixels = (np.right_shift(bit_image,5)&1).sum()
    snow_ratio = snow_pixels / bit_image.size
    if snow_ratio < 0.05:
        return 0
    elif snow_ratio < 0.1:
        return 1
    elif snow_ratio < 0.3:
        return 2
    else:
        return 3

for path, subdirs, files in os.walk(src_dir):
    for name in files:
        cloud_mod, snow_mod = 0,0
        if name.endswith(".npy"):
            bit_image = np.load(os.path.join(path, name))
            cloud_mod = get_cloud_segment(bit_image)
            snow_mod = get_snow_segment(bit_image)
            caption = caption_prompt.format(cloud=cloud_modifiers[cloud_mod], snow=snow_modifiers[snow_mod])
            captions.append(caption)

with open(os.path.join(target_dir, 'captions.txt'), 'w') as f:
    for caption in captions:
        f.write(caption + '\n')

            


131